In [9]:
using JuMP
using Ipopt
using CSV
using GraphRecipes, Plots
using LightGraphs
using LinearAlgebra
using SparseArrays
using DataFrames
using Juniper, Alpine

In [10]:
innovations = CSV.File("./innovations.csv",header=1,delim=",") |> DataFrame
mapki = CSV.File("./map-ki.csv",header=1,delim=",") |> DataFrame
compromise = CSV.File("./compromise.csv",header=1,delim=",") |> DataFrame
initial = CSV.File("./initial.csv",header=1,delim=",") |> DataFrame


,k,i,GWP,Theta,Per_Recycl,Per_Comp
,Int64,Int64,Int64,Float64,Float64,Int64
1,0,0,100,0.25,0.12,0


In [11]:
i=5
p=i-4
print(p)

1

In [19]:
function tech_roadmap()
    rdmap=Model(Alpine.Optimizer)
    @variable(rdmap,yk[k=1:nrow(compromise)],Bin)
    @variable(rdmap,0 ≤ tk[k=1:nrow(compromise)] ≤ 50)
    @variable(rdmap,0 ≤ xk[k=1:nrow(compromise)] ≤ 1)
    @variable(rdmap,yi[i=1:nrow(innovations)],Bin)
    @variable(rdmap,0 ≤ ti[i=1:nrow(innovations)] ≤ 50)
    
    @constraint(rdmap, [i=1], sum(yk[k] for k=1:mapki[i,2])==yi[i])
    @constraint(rdmap, [i=2:nrow(mapki)], sum(yk[k] for k=mapki[i-1,2]:mapki[i,2])==yi[i])
    
    @constraint(rdmap, [i=1,k=1:mapki[i,2]], ti[i] ≤ tk[k])
    @constraint(rdmap, [i=2:nrow(mapki),k=mapki[i-1,2]:mapki[i,2]], ti[i] ≤ tk[k])
    
    @constraint(rdmap, [i=1:nrow(mapki)], 50*yi[i] ≥ ti[i])
    @constraint(rdmap, [k=1:nrow(compromise)], 50*yk[k] ≥ tk[k])
    
    @constraint(rdmap, [k=1:nrow(compromise)], yk[k] ≥ xk[k])
    @constraint(rdmap, sum(xk[k] for k in 1:nrow(compromise)) ≤ 1) 
    
    
    dummy=10
    @objective(rdmap,Min,dummy)
    
    optimize!(rdmap)
    return JuMP.value.(yk)  
end

tech_roadmap (generic function with 1 method)

In [20]:
y=tech_roadmap()

LoadError: MethodError: no method matching initialize(::Nothing, ::Vector{Symbol})
[0mClosest candidates are:
[0m  initialize([91m::MathOptInterface.Test.HS071[39m, ::Vector{Symbol}) at /home/thakker.8/.julia/packages/MathOptInterface/YDdD3/src/Test/nlp.jl:22
[0m  initialize([91m::MathOptInterface.Test.FeasibilitySenseEvaluator[39m, ::Vector{Symbol}) at /home/thakker.8/.julia/packages/MathOptInterface/YDdD3/src/Test/nlp.jl:280
[0m  initialize([91m::NLPEvaluator[39m, ::Vector{Symbol}) at /home/thakker.8/.julia/packages/JuMP/klrjG/src/nlp.jl:411
[0m  ...